In [2]:
import json
import pandas as pd
import re

from collections import Counter
# from collections import defaultdict
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import normalize
import math

from math import log

In [3]:
input_file_name = './data/extractive_test_v2.jsonl'

In [4]:
def preprocess_sentence(article):
    article = re.sub('”', ' ', article)
#     article = re.sub('  ', ' ', article)
#     article = re.sub('[-=+,#/\?:^$.@*※~&%ㆍ!』\\‘|\[\]\<\>`\'…》]', '', article)
    bracket = re.findall(r'\([^)]*\)', article )
    for i in bracket:
        word = i.strip('()')
        if word.isupper():
            end_index = article.find(i)
            word_len = article[end_index:0:-1].find(' ')
            start_index = end_index - word_len +1
            origin = article[start_index : end_index]
            article = article[:end_index+len(i)] + article[end_index+len(i):].replace(word, origin)
        else:
            if '이하' in word:
                word = word[3:]
                n_space = word.count(' ')
                end_index = article.find(word)-4
                range_candidate = article[end_index-30:end_index].split(' ')[::-1]
                origin = ' '.join(range_candidate[:n_space+1][::-1])
                article = article[:end_index+len(i)] + article[end_index+len(i):].replace(word, origin)
        # 괄호는 다 제거
        article = article.replace(i,'')
#         article = article.replace('당진','')
    article = ''.join(re.findall('[ 가-힣a-zA-Z0-9]',  article) )
    
    return article.strip(' ')

In [5]:
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w or '/SN' in w)]
    return words

In [35]:
def scan_vocabulary(sents, tokenize, min_count):
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w:c for w,c in counter.items() if c >= min_count}
    
    return counter

In [45]:
df_value = pd.DataFrame(columns = ['frequency'])

In [51]:
with open(input_file_name, 'r', encoding = 'utf-8', newline = '') as input_file:
    i = 0
    for line in input_file:
        line = json.loads(line)
        id_num, sents = list(line.values())[1:]
        preprocessed = [ preprocess_sentence(sent) for sent in sents ]
        counter = scan_vocabulary(preprocessed , komoran_tokenize , 2)
        
        for j in counter.keys():
            if j in df_value.index:
                df_value.loc[j] += counter[j]
            else:
                df_value.loc[j] = counter[j]
        
        i += 1
        if i % 100 == 0 :
            df_value = df_value[df_value['frequency'] / i  > 0.3]
    print(i)

C:\Users\Playdata\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


9987


In [52]:
# df_value = df_value[df_value['frequency'] > 1000 ]
df_value

,frequency
있/VV,75192
것/NNB,28992
년/NNB,20896
수/NNB,28130
일/NNB,29624
...,...
관리/NNG,3
모션/NNP,3
회원사/NNG,2
스마트/NNP,2


In [57]:
len(df_value[df_value['frequency'] > 10 ])

262

In [59]:
df2 = df_value[df_value['frequency'] > 100 ]
df2.index

,frequency
있/VV,75192
것/NNB,28992
년/NNB,20896
수/NNB,28130
일/NNB,29624
위하/VV,21942
등/NNB,55394
하/VV,35248
원/NNB,7852
지나/VV,3861


In [60]:
df2.index

Index(['있/VV', '것/NNB', '년/NNB', '수/NNB', '일/NNB', '위하/VV', '등/NNB', '하/VV',
       '원/NNB', '지나/VV', '2/SN', '시/NNB', '운영/NNG', '되/VV', '3/SN', '1/SN',
       '대하/VV', '말/NNG', '명/NNB', '사업/NNG', '지역/NNG', '추진/NNG', '통하/VV',
       '주민/NNG', '받/VV'],
      dtype='object')